<a href="https://colab.research.google.com/github/aashmauprety/Machine-Learning/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
url = 'https://raw.githubusercontent.com/aashmauprety/Machine-Learning/master/Churn_Modelling.csv'

In [0]:
dataset = pd.read_csv(url)

In [0]:
print(dataset)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
#dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [27]:
# Importing the Keras libraries and packages
# Part 2 - Now let's make the ANN! 

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

classifier = Sequential() # Initialising the ANN
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # Adding the second hidden layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # Adding the output layer
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # Fitting the ANN to the Training set


Epoch 1/100
8000/8000 [==============================] - 1s 136us/step - loss: 0.5031 - accuracy: 0.7972
Epoch 2/100
8000/8000 [==============================] - 1s 126us/step - loss: 0.4102 - accuracy: 0.8256
Epoch 3/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.3960 - accuracy: 0.8292
Epoch 4/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.3878 - accuracy: 0.8307
Epoch 5/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.3810 - accuracy: 0.8311
Epoch 6/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.3759 - accuracy: 0.8400
Epoch 7/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.3723 - accuracy: 0.8450
Epoch 8/100
8000/8000 [==============================] - 1s 124us/step - loss: 0.3693 - accuracy: 0.8484
Epoch 9/100
8000/8000 [==============================] - 1s 129us/step - loss: 0.3661 - accuracy: 0.8490
Epoch 10/100
8000/8000 [==============================]

In [0]:
y_predict = classifier.predict(X_test)
y_predict = (y_predict > 0.5)
print(y_predict)

In [0]:
#Cross Validate
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
 classifier = Sequential() # Initialising the ANN
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) # Adding the input layer and the first hidden layer
 classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu')) # Adding the second hidden layer
 classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')) # Adding the output layer
 classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Compiling the ANN
 #classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # Fitting the ANN to the Training set
 return classifier

classifier = KerasClassifier(build_fn= build_classifier, batch_size= 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X= X_train, y = y_train, cv = 10, n_jobs= 1)

In [0]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

In [0]:
#Parameters Tuning for improving accuracy 
from keras.layers import Dropout